# LangChain

## 4-1. LangChain 개요

In [35]:
# !pip install langchain openai langchain-openai

In [36]:
# !pip install langchain_classic

In [37]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(len(OPENAI_API_KEY))

164


## 4-2. Language Models

### LLM

In [38]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
result = llm.invoke("자기소개를 해주세요.")
print(result)

content='안녕하세요! 저는 OpenAI에서 개발한 AI 언어 모델, ChatGPT입니다. 다양한 주제에 대해 대화하고 질문에 답변하며 글쓰기, 번역, 학습 도움 등 여러 가지 작업을 도와드릴 수 있습니다. 궁금한 점이나 도움이 필요하시면 언제든지 말씀해 주세요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 14, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_376a7ccef1', 'id': 'chatcmpl-CzvHaDmHSxL0cVpGBpYqxReHF6F3q', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bd91e-dfeb-7863-8728-475559a857a0-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 14, 'output_tokens': 68, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Chat Models

In [39]:
from langchain_openai import ChatOpenAI
from langchain_classic.schema import AIMessage, HumanMessage, SystemMessage
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요! 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존 씨! 어떻게 도와드릴까요?"),
    HumanMessage(content= "제 이름을 아세요?")
]
result = chat.invoke(messages)
print(result.content)

네, 존 씨라고 하셨어요! 무엇을 도와드릴까요?


### Callback을 이용한 스트리밍
**StreamingStdOutCallbackHandler**
- 랭체인(LangChain)에서 LLM이 답변을 생성할 때, 전체 답변이 완성될 때까지 기다리지 않고 생성되는 즉시 한 글자(토큰)씩 화면에 출력해주는 도구
- 가장 중요한 점은 LLM 객체를 생성할 때 **streaming=True** 설정을 반드시 넣어줘야 한다

In [40]:
from langchain_classic.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_classic.schema import HumanMessage
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model_name="gpt-4.1-mini",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
messages = [HumanMessage(content="자기소개를 해주세요")]
result = chat.invoke(messages)

안녕하세요! 저는 OpenAI에서 개발한 AI 언어 모델, ChatGPT입니다. 다양한 주제에 대해 대화하고 질문에 답변하며 글쓰기, 번역, 학습 도움 등 여러 가지 작업을 도와드릴 수 있습니다. 궁금한 점이나 도움이 필요하시면 언제든지 말씀해 주세요!

## 4-3. Prompts

### Prompt templates

In [41]:
from langchain_classic.prompts import PromptTemplate
template = """
다음 요리의 레시피를 생각해 주세요.
요리: {dish}
"""
prompt = PromptTemplate(input_variables=["dish"], template=template)
result = prompt.format(dish="카레")
print(result)


다음 요리의 레시피를 생각해 주세요.
요리: 카레



### ChatPromptTemplate

In [42]:
from langchain_classic.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_classic.schema import HumanMessage, SystemMessage
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("당신은 {country} 요리 전문가입니다."),
    HumanMessagePromptTemplate.from_template("다음 요리의 레시피를 생각해 주세요.\n\n요리: {dish}")
])
messages = chat_prompt.format_prompt(country="영국", dish="고기감자조림").to_messages()
print(type(messages))
print(messages)

<class 'list'>
[SystemMessage(content='당신은 영국 요리 전문가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='다음 요리의 레시피를 생각해 주세요.\n\n요리: 고기감자조림', additional_kwargs={}, response_metadata={})]


In [43]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
result = chat.invoke(messages)
print(type(result), type(result.content))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'> <class 'str'>
고기감자조림은 한국 가정식이지만, 영국식 감자 요리와도 잘 어울리는 맛있는 조림 요리입니다. 영국 요리 전문가로서 약간의 영국식 터치를 가미한 고기감자조림 레시피를 소개해 드리겠습니다.

### 고기감자조림 (Braised Beef and Potatoes)

#### 재료 (4인분)
- 소고기 (국거리용 또는 찜용) 400g, 한 입 크기로 썰기
- 감자 4~5개, 껍질 벗기고 큼직하게 깍둑썰기
- 양파 1개, 채썰기
- 당근 1개, 큼직하게 썰기 (선택 사항)
- 마늘 3쪽, 다진 것
- 생강 1 작은술, 다진 것 (선택 사항)
- 간장 4 큰술
- 설탕 1 큰술
- 우스터 소스 1 큰술 (영국식 감칠맛 추가)
- 소금, 후추 약간
- 식용유 2 큰술
- 물 300ml
- 파슬리 약간 (마무리용)

#### 조리 방법
1. 큰 팬이나 냄비에 식용유를 두르고 중불에서 다진 마늘과 생강을 볶아 향을 낸다.
2. 소고기를 넣고 겉면이 갈색이 될 때까지 볶는다.
3. 양파와 당근을 넣고 2~3분간 함께 볶아준다.
4. 감자를 넣고 간장, 설탕, 우스터 소스, 소금, 후추를 넣어 간을 맞춘다.
5. 물을 부어 재료가 잠길 정도로 맞추고 끓인다.
6. 끓기 시작하면 중약불로 줄이고 뚜껑을 덮어 30~40분간 감자가 부드러워질 때까지 조린다.
7. 국물이 너무 많으면 뚜껑을 열고 중불에서 졸여 농도를 맞춘다.
8. 완성된 고기감자조림에 다진 파슬리를 뿌려 장식한다.

#### 팁
- 우스터 소스는 영국 요리에 자주 쓰이는 소스로, 감칠맛과 깊은 풍미를 더해줍니다.
- 감자는 영국산 마를로즈(Marvel of the White)나 로즈마리 감자와 같은 품종을 사용하면 더욱 맛있습니다.
- 고기 대신 양고기를 사용해도 좋습니다.

맛있게 즐기세요!


## 4-4. Output Parsers

### PydanticOutputParser

In [44]:
from pydantic import BaseModel, Field
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [45]:
from langchain_classic.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=Recipe)

In [46]:
format_instructions = parser.get_format_instructions()
print(type(format_instructions))
print(format_instructions)

<class 'str'>
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [47]:
from langchain_classic.prompts import PromptTemplate
template = """다음 요리의 레시피를 한국어로 생각해 주세요.
{format_instructions}
요리: {dish}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": format_instructions}
)
formatted_prompt = prompt.format(dish="카레")
print(formatted_prompt)

다음 요리의 레시피를 한국어로 생각해 주세요.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
요리: 카레



In [48]:
from langchain_openai import ChatOpenAI
from langchain_classic.schema import HumanMessage
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
messages = [HumanMessage(content=formatted_prompt)]
output = chat.invoke(messages)
print(output.content)

```json
{
  "ingredients": [
    "닭고기 300g",
    "감자 2개",
    "당근 1개",
    "양파 1개",
    "카레 가루 3큰술",
    "식용유 2큰술",
    "물 500ml",
    "소금 약간",
    "후추 약간"
  ],
  "steps": [
    "닭고기는 한 입 크기로 썰고, 감자와 당근은 껍질을 벗겨 큼직하게 자른다.",
    "양파는 채 썬다.",
    "냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 익힌다.",
    "닭고기를 넣고 겉면이 하얗게 변할 때까지 볶는다.",
    "감자와 당근을 넣고 함께 볶는다.",
    "물 500ml를 붓고 끓기 시작하면 중약불로 줄여 재료가 부드러워질 때까지 약 15분간 끓인다.",
    "카레 가루를 넣고 잘 저어가며 5분 정도 더 끓인다.",
    "소금과 후추로 간을 맞춘 후 불을 끄고 잠시 둔다.",
    "밥과 함께 접시에 담아 낸다."
  ]
}
```


In [49]:
recipe = parser.parse(output.content)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 300g', '감자 2개', '당근 1개', '양파 1개', '카레 가루 3큰술', '식용유 2큰술', '물 500ml', '소금 약간', '후추 약간'] steps=['닭고기는 한 입 크기로 썰고, 감자와 당근은 껍질을 벗겨 큼직하게 자른다.', '양파는 채 썬다.', '냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 익힌다.', '닭고기를 넣고 겉면이 하얗게 변할 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '물 500ml를 붓고 끓기 시작하면 중약불로 줄여 재료가 부드러워질 때까지 약 15분간 끓인다.', '카레 가루를 넣고 잘 저어가며 5분 정도 더 끓인다.', '소금과 후추로 간을 맞춘 후 불을 끄고 잠시 둔다.', '밥과 함께 접시에 담아 낸다.']


## 4-5. Chains

### LLMChain : PromptTemplate, Language model, OutputParser를 연결

In [50]:
from langchain_openai import ChatOpenAI
from langchain_classic.output_parsers import PydanticOutputParser
from langchain_classic.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)
template = """다음 요리의 레시피를 한국어로 생각해 주세요.
{format_instructions}
요리: {dish}
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["dish"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

In [51]:
from langchain_classic.chains import LLMChain
chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)
recipe = chain.invoke("카레")
print(type(recipe))
print(recipe)

<class 'dict'>
{'dish': '카레', 'text': Recipe(ingredients=['닭고기 300g', '감자 2개', '당근 1개', '양파 1개', '카레 가루 3큰술', '식용유 2큰술', '물 500ml', '소금 약간', '후추 약간'], steps=['닭고기는 한 입 크기로 썰고, 감자와 당근은 껍질을 벗겨 큼직하게 자른다.', '양파는 채 썬다.', '냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 익힌다.', '닭고기를 넣고 겉면이 하얗게 변할 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '물 500ml를 붓고 끓기 시작하면 중약불로 줄여 재료가 부드러워질 때까지 약 15분간 끓인다.', '카레 가루를 넣고 잘 저어가며 5분 정도 더 끓인다.', '소금과 후추로 간을 맞춘 후 불을 끄고 잠시 둔다.', '밥과 함께 접시에 담아 낸다.'])}


### SimpleSequentialChain ― Chain과 Chain을 연결

In [61]:
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
#
cot_template = """다음 질문에 답하세요.
질문: {question}
단계별로 생각해 봅시다.
"""
cot_prompt = PromptTemplate(input_variables=["question"], template=cot_template)
cot_chain = LLMChain(llm=chat, prompt=cot_prompt)
#
summarize_template = """다음 문장을 결론만 간단히 요약하세요.
{input}
"""
summarize_prompt = PromptTemplate(input_variables=["input"], template=summarize_template)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [62]:
from langchain_classic.chains import SimpleSequentialChain
cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])
result = cot_summarize_chain.invoke(
"""저는 시장에 가서 사과 10개를 샀습니다.
이웃에게 2개, 수리공에게 2개를 주었습니다.
그런 다음에 사과 5개를 더 사서 1개를 먹었습니다.
남은 개수는 몇 개인가요?"""
)
print(result["output"])

최종적으로 남은 사과는 10개입니다.


## 4-6. Memory

### ConversationBufferMemory

In [64]:
from langchain_classic.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain_classic.memory import ConversationBufferMemory

chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

while True:
    user_message = input("You: ")
    print(user_message)
    if user_message == "끝":
        print("(대화 종료)")
        break
    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

나는 KSH야. 너는?
AI: 안녕하세요, KSH님! 저는 AI 어시스턴트예요. 여러분과 대화하고 도움을 드리기 위해 만들어졌답니다. 무엇이든 궁금한 점이나 이야기하고 싶은 주제가 있으면 언제든지 말씀해 주세요!
바이브코딩을 잘하는 방법은?
AI: 바이브코딩(Vibe Coding)을 잘하는 방법에 대해 알려드릴게요! 바이브코딩은 보통 프로그래밍을 할 때 자신만의 리듬과 몰입 상태를 찾아 효율적으로 코딩하는 방식을 의미해요. 이를 잘하기 위한 몇 가지 팁을 드리자면:

1. **환경 조성하기**  
   자신에게 맞는 조용하고 편안한 작업 공간을 만드는 것이 중요해요. 조명이 너무 밝거나 어두우면 집중력이 떨어질 수 있으니 적절한 조명을 유지하고, 잡음이 적은 곳에서 작업하세요.

2. **음악 활용하기**  
   많은 개발자들이 집중력을 높이기 위해 백색소음이나 Lo-fi, 클래식 음악 등을 듣곤 해요. 자신에게 맞는 음악을 찾아보세요. 너무 가사 있는 음악은 오히려 방해가 될 수 있어요.

3. **목표 설정과 시간 관리**  
   하루에 달성할 작은 목표를 세우고, 타이머(예: 포모도로 기법)를 활용해 일정 시간 집중 후 짧은 휴식을 취하는 방법이 좋아요. 이렇게 하면 지치지 않고 꾸준히 몰입할 수 있어요.

4. **코딩 스타일 정립하기**  
   자신만의 코딩 스타일과 규칙을 만들어두면 코딩할 때 생각이 덜 분산되고 더 빠르게 진행할 수 있어요. 예를 들어, 변수명 짓는 규칙, 함수 작성 방식 등을 미리 정해두는 거죠.

5. **꾸준한 연습과 복습**  
   바이브코딩은 단순히 빠르게 코딩하는 게 아니라, 자신만의 리듬을 찾는 과정이에요. 꾸준히 코딩하면서 어떤 방식이 자신에게 가장 잘 맞는지 실험하고, 그 경험을 바탕으로 개선해 나가세요.

6. **건강 관리**  
   충분한 수면과 규칙적인 운동도 집중력과 몰입에 큰 영향을 줘요. 몸이 건강해야 머리도 잘 돌아가니까요.

혹시 바이브코딩과 관련해서 구체적인 도구나 기술, 혹은 특정 프로그

### Chat models에서 Memory를 사용할 때 주의할 점

In [60]:
from langchain_openai import ChatOpenAI
from langchain_classic.schema import AIMessage, HumanMessage, SystemMessage
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="안녕하세요. 저는 존이라고 합니다!"),
    AIMessage(content="안녕하세요, 존 님! 어떻게 도와드릴까요?"),
    HumanMessage(content= "제 이름을 아세요?")
]
result = chat.invoke(messages)
print(result.content)

네, 존 님이라고 하셨어요! 무엇을 도와드릴까요?
